Pakete laden

In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from shapely.geometry import Point

In [2]:
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None

Funktionen

In [3]:
# Funktion, um den Index der nächstgelegenen Wetterstation zu finden
def find_nearest_station_index(point, stations):
    point = gpd.GeoDataFrame(geometry=[point], crs=stations.crs)
    distances = stations.geometry.apply(lambda station: point.distance(station).values[0])
    return distances.idxmin()

Standorte von Staumauern laden

In [4]:
locations = pd.read_csv(r"C:\Users\fth\GovTechHackathon2024-Schneereserven\tol\standorte.csv")

In [5]:
locations['geometry'] =  locations['geometry'].apply(wkt.loads)
point_gdf = gpd.GeoDataFrame(data=locations, geometry="geometry")
# Koordinaten extrahieren
point_gdf['X'] = point_gdf['geometry'].x
point_gdf['Y'] = point_gdf['geometry'].y

Wetterstationen laden

In [6]:
# URL des CSV
url = "https://data.geo.admin.ch/ch.meteoschweiz.messnetz-automatisch/ch.meteoschweiz.messnetz-automatisch_de.csv"

# CSV direkt als DataFrame laden
weather_stations_df = pd.read_csv(url, encoding="latin-1", sep=";")
# Zielen entfernen, bei denen "Daten seit" NULL ist
weather_stations_df = weather_stations_df[weather_stations_df['Daten seit'].notnull()]

In [7]:
weather_stations_gdf = gpd.GeoDataFrame(
    weather_stations_df,
    geometry=gpd.points_from_xy(weather_stations_df.KoordinatenE, weather_stations_df.KoordinatenN),
    crs="EPSG:2056"  # LV95 Koordinatensystem
)
weather_stations_gdf.head()

,Station,Abk.,WIGOS-ID,Stationstyp,Eigentümer,Daten seit,Stationshöhe m ü. M.,Barometerhöhe m ü. Boden,KoordinatenE,KoordinatenN,Breitengrad,Längengrad,Exposition,Kanton,Messungen,Link,geometry
0,Aadorf / Tänikon,TAE,0-20000-0-06679,Wetterstation,MeteoSchweiz,01.01.1970,539.0,539.0,2710518.0,1259824.0,47.479892,8.904928,Ebene,TG,"Temperatur, Feuchte, Taupunkt, Niederschlag, Druck, Wind, Globalstrahlung, Sonnenschein, Schnee, Temperatur 5cm, Bodentemperatur, Strahlung langwellig",https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&station=TAE,POINT (2710518.000 1259824.000)
1,Aarberg,ABE,0-756-0-ABE,Niederschlagsstation,MeteoSchweiz,01.10.1900,444.0,NaN,2588354.0,1211894.0,47.057969,7.285350,Ebene,BE,"Niederschlag, Schnee",https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&station=ABE,POINT (2588354.000 1211894.000)
2,Acquarossa / Comprovasco,COM,0-20000-0-06756,Wetterstation,MeteoSchweiz,01.10.1900,575.0,576.0,2714985.0,1146451.0,46.459517,8.935486,Tal,TI,"Temperatur, Feuchte, Taupunkt, Niederschlag, Druck, Wind, Globalstrahlung, Sonnenschein, Föhnindex, Temperatur 5cm",https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&station=COM,POINT (2714985.000 1146451.000)
3,Adelboden,ABO,0-20000-0-06735,Wetterstation,"MeteoSchweiz, SLF",01.01.1901,1321.0,1326.0,2609372.0,1148939.0,46.491703,7.560703,Südosthang,BE,"Temperatur, Feuchte, Taupunkt, Niederschlag, Druck, Wind, Globalstrahlung, Sonnenschein, Schnee, Föhnindex, Temperatur 5cm",https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&station=ABO,POINT (2609372.000 1148939.000)
4,Affoltern i. E.,AIE,0-756-0-AIE,Niederschlagsstation,MeteoSchweiz,01.01.1864,755.0,NaN,2622940.0,1212500.0,47.063125,7.740614,Osthang,BE,Niederschlag,https://www.meteoschweiz.admin.ch/service-und-publikationen/applikationen/messwerte-und-messnetze.html#param=messnetz-automatisch&station=AIE,POINT (2622940.000 1212500.000)


In [8]:
# Ermitteln der Indexwerte der nächstgelegenen Stationen
point_gdf['nearest_station_index'] = point_gdf.geometry.apply(lambda x: find_nearest_station_index(x, weather_stations_gdf))

In [9]:
# Hinzufügen der Informationen der nächstgelegenen Stationen zu point_gdf
for column in weather_stations_gdf.columns:
    point_gdf[column] = point_gdf['nearest_station_index'].apply(lambda idx: weather_stations_gdf.loc[idx, column])

# Entfernen der Hilfsspalte
point_gdf.drop(columns=['nearest_station_index'], inplace=True)